# Chat with Data Notebook (Embedded Tables)

In this notebook we walk you through an advanced RAG use case - parsing a complex document with embedded tables - and how to handle that in LlamaIndex.

We compare our approach against a "naive" RAG stack (using top-k retrieval with a fixed chunk size).



In [ ]:
!pip install llama-index llama-hub pypdf --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.9/103.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.8/176.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━

In [ ]:
# setup OpenAI
import openai

openai.api_key = ""

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

## Complex Document (with Embedded Tables)

In this setting we walk through a document that has an embedded table inside of it.

In [ ]:
!apt install ghostscript python3-tk

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-tk is already the newest version (3.10.8-1~22.04).
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 libgs9 libgs9-common libidn12 libijs-0.35
  libjbig2dec0 poppler-data
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre ghostscript-x poppler-utils
  fonts-japanese-mincho | fonts-ipafont-mincho fonts-japanese-gothic | fonts-ipafont-gothic
  fonts-arphic-ukai fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript libgs9 libgs9-common libidn12
  libijs-0.35 libjbig2dec0 poppler-data
0 upgraded, 10 newly installed, 0 to remove and 49 not upgraded.
Need to get 16.7 MB of archives.
After this operation, 63.0 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 

In [ ]:
!pip install camelot-py pymupdf frontend ghostscript

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 97.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 108.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 6.2 MB/s eta 0:00:00


In [ ]:
# aaron 1
!pip install matplotlib-venn

In [ ]:
!apt-get update

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Ign:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:11 https://r2u.stat.illinois.edu/ubuntu jammy Release
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:
!apt-get update --fix-missing

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Ign:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:11 https://r2u.stat.illinois.edu/ubuntu jammy Release
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:
!apt-get upgrade

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Calculating upgrade... Done
The following packages were automatically installed and are no longer required:
  linux-headers-5.15.0-122 linux-headers-5.15.0-122-generic
Use 'apt autoremove' to remove them.
The following packages have been kept back:
  libcudnn8 libcudnn8-dev libnccl-dev libnccl2
0 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.


In [ ]:
# aaron 2
!apt-get -qq install -y libfluidsynth1

E: Package 'libfluidsynth1' has no installation candidate


In [ ]:
# aaron
!apt-get install update

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package update


In [ ]:
# aaron
!apt-get install upgrade

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package upgrade


In [ ]:
# aaron
!apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,032 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy I

In [ ]:
!apt upgrade

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Calculating upgrade... Done
The following packages were automatically installed and are no longer required:
  linux-headers-5.15.0-122 linux-headers-5.15.0-122-generic
Use 'apt autoremove' to remove them.
The following NEW packages will be installed:
  linux-headers-5.15.0-124 linux-headers-5.15.0-124-generic
The following packages have been kept back:
  libcudnn8 libcudnn8-dev libnccl-dev libnccl2
The following packages will be upgraded:
  base-files bash binutils binutils-common binutils-x86-64-linux-gnu bsdutils coreutils
  cuda-compat-12-2 cuda-keyring cuda-toolkit-12-config-common cuda-toolkit-config-common dpkg
  dpkg-dev e2fsprogs libarchive13 libbinutils libblkid1 libc-bin libctf-nobfd0 libctf0 libdpkg-perl
  libext2fs2 libgnutls30 libldap-2.5-0 libmount1 libpam-modules libpam-modules-bin libpam-runtime
  libpam0g libperl5.34 libprocps8 libsmartcols1 libss2 libudev1 linux-headers-ge

In [ ]:
# https://pypi.python.org/pypi/libarchive
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
import libarchive

  Using cached libarchive-0.4.7.tar.gz (23 kB)
  Preparing metadata (setup.py) ... done
  Using cached nose-1.3.7-py3-none-any.whl.metadata (1.7 kB)
Using cached nose-1.3.7-py3-none-any.whl (154 kB)
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for libarchive
  Running setup.py clean for libarchive
Failed to build libarchive
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (libarchive)


ModuleNotFoundError: No module named 'libarchive'

In [ ]:
# https://pypi.python.org/pypi/libarchive
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
import libarchive

In [ ]:
# aaron 3
# https://pypi.python.org/pypi/libarchive
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
import libarchive

  Using cached libarchive-0.4.7.tar.gz (23 kB)
  Preparing metadata (setup.py) ... done
  Using cached nose-1.3.7-py3-none-any.whl.metadata (1.7 kB)
Using cached nose-1.3.7-py3-none-any.whl (154 kB)
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for libarchive
  Running setup.py clean for libarchive
Failed to build libarchive
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (libarchive)


ModuleNotFoundError: No module named 'libarchive'

In [ ]:
# aaron 4
# https://pypi.python.org/pypi/pydot
!apt-get -qq install -y graphviz && pip install pydot
import pydot

In [ ]:
# aaron 5
!pip install cartopy
import cartopy

In [ ]:
# aaron new
%pip install llama-index-embeddings-openai
%pip install llama-index-readers-file pymupdf
%pip install llama-index-llms-openai
%pip install llama-index-experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.6/187.6 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.2/249.2 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.7/229.7 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.4/197.4 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.1/113.1 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 112.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# aaron new

import camelot

# https://en.wikipedia.org/wiki/The_World%27s_Billionaires
from llama_index.core import VectorStoreIndex
from llama_index.experimental.query_engine import PandasQueryEngine
from llama_index.core.schema import IndexNode
from llama_index.llms.openai import OpenAI

from llama_index.readers.file import PyMuPDFReader
from typing import List

In [ ]:
import camelot
from llama_index import Document, SummaryIndex

# https://en.wikipedia.org/wiki/The_World%27s_Billionaires
from llama_index import VectorStoreIndex, ServiceContext, LLMPredictor
from llama_index.query_engine import PandasQueryEngine, RetrieverQueryEngine
from llama_index.retrievers import RecursiveRetriever
from llama_index.schema import IndexNode
from llama_index.llms import OpenAI

from llama_hub.file.pymu_pdf.base import PyMuPDFReader
from pathlib import Path
from typing import List

ImportError: cannot import name 'Document' from 'llama_index' (unknown location)

### Parse out Table, build Pandas Query Engine

In [ ]:
!wget "https://www.dropbox.com/scl/fi/waoz9bo9yiemnhnqvu0cc/billionaires_page.pdf?rlkey=4i08msa7zr1lpnuq2y1vs2xgw&dl=1" -O billionaires_page.pdf

--2023-11-08 08:39:04--  https://www.dropbox.com/scl/fi/waoz9bo9yiemnhnqvu0cc/billionaires_page.pdf?rlkey=4i08msa7zr1lpnuq2y1vs2xgw&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.67.18, 2620:100:6025:18::a27d:4512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.67.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc19051c67f1e7f1353ff651529f.dl.dropboxusercontent.com/cd/0/inline/CHILPsnmWsPf2A36gkigVOpcEGOEPC5ozf159kcAuew7RElvDPzsnCSDtWgUAhofhJWPHM7BUp-qrES7oGQnSWKUQBGFH126pEgeN72xQsYTBjw4h5Yq56h2KBJ5w3Rrx20/file?dl=1# [following]
--2023-11-08 08:39:04--  https://uc19051c67f1e7f1353ff651529f.dl.dropboxusercontent.com/cd/0/inline/CHILPsnmWsPf2A36gkigVOpcEGOEPC5ozf159kcAuew7RElvDPzsnCSDtWgUAhofhJWPHM7BUp-qrES7oGQnSWKUQBGFH126pEgeN72xQsYTBjw4h5Yq56h2KBJ5w3Rrx20/file?dl=1
Resolving uc19051c67f1e7f1353ff651529f.dl.dropboxusercontent.com (uc19051c67f1e7f1353ff651529f.dl.dropboxusercontent.com)... 162.125.71.15, 2620:100:6021:15::a2

In [ ]:
!cp /Users/aaron/ws/"# 2024"/"70 菱光社 奇美LLM開發維運/單張/2023表1.pdf" .

cp: cannot stat '/Users/aaron/ws/# 2024/70 菱光社 奇美LLM開發維運/單張/2023表1.pdf': No such file or directory


In [ ]:
!cp /Users/aaron/ws/2023表1.pdf .

cp: cannot stat '/Users/aaron/ws/2023表1.pdf': No such file or directory


In [ ]:
# initialize PDF reader
reader = PyMuPDFReader()

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!ls /content/drive/MyDrive/'Colab Notebooks'/


 2023年度合併財務報表.pdf   2023表4.pdf
 2023文表1-5.pdf	    ah_openai_embed_test01.ipynb
 2023表1.pdf		   'Copy of Chat with Data Notebook (Embedded Tables, extended)'
 2023表2.pdf		    ngrok_test.ipynb
 2023表3.pdf		    openai_test02.ipynb


In [ ]:
!ls /content/drive/MyDrive/'Colab Notebooks'/2023表1.pdf

'/content/drive/MyDrive/Colab Notebooks/2023表1.pdf'


In [ ]:
#file_path = "billionaires_page.pdf"
file_path = '/content/drive/MyDrive/Colab Notebooks/2023表1.pdf'
docs = reader.load(file_path=file_path)

In [ ]:
# use camelot to parse tables
def get_tables(path: str, pages: List[int]):
    table_dfs = []
    for page in pages:
        table_list = camelot.read_pdf(path, pages=str(page))
        table_df = table_list[0].df
        table_df = (
            table_df.rename(columns=table_df.iloc[0])
            .drop(table_df.index[0])
            .reset_index(drop=True)
        )
        table_dfs.append(table_df)
    return table_dfs

In [ ]:
table_dfs = get_tables(file_path, pages=[1, 1])

IndexError: list index out of range

In [ ]:
# aaron test:
table_list = camelot.read_pdf(file_path, pages="1")
print(f"Number of tables found: {len(table_list)}")

Number of tables found: 0


In [ ]:
!ls '/content/drive/MyDrive/Colab Notebooks/'


 2023年度合併財務報表.pdf   2023表4.pdf
 2023文表1-5.pdf	    ah_openai_embed_test01.ipynb
 2023表1.pdf		   'Copy of Chat with Data Notebook (Embedded Tables, extended)'
 2023表2.pdf		    ngrok_test.ipynb
 2023表3.pdf		    openai_test02.ipynb


In [ ]:
# file_path = '/content/drive/MyDrive/Colab Notebooks/2023表1.pdf'
# file_path = '/content/drive/MyDrive/Colab Notebooks/2023表2.pdf'
# file_path = '/content/drive/MyDrive/Colab Notebooks/2023表3.pdf'
file_path = '/content/drive/MyDrive/Colab Notebooks/2023表4.pdf'
# file_path = '/content/drive/MyDrive/Colab Notebooks/2023文表1-5.pdf'
# file_path = '/content/drive/MyDrive/Colab Notebooks/2023年度合併財務報表.pdf'
# aaron test:
table_list = camelot.read_pdf(file_path, pages="1")
print(f"Number of tables found: {len(table_list)}")

Number of tables found: 0


In [ ]:
file_path = '/content/drive/MyDrive/Colab Notebooks/2023表4.pdf'

table_list = camelot.read_pdf(file_path, pages="1", flavor="stream")


In [ ]:
table_list[0]

<Table shape=(39, 4)>

In [ ]:
for i, j in table_list:
  print(table_list[i,j])

TypeError: cannot unpack non-iterable Table object

In [ ]:
#aaron
# Get the table as a pandas DataFrame
table_df = table_list[0].df

# Print the DataFrame
print(table_df)


                       0  1            2            3
0                                            單位：新台幣千元
1                                  112年度        111年度
2             營業活動之現金流量：                             
3              本期稅前淨(損)利  $  (1,112,021)    5,558,149
4                  調整項目：                             
5                 收益費損項目                             
6                   折舊費用       5,022,749    4,541,542
7                   攤銷費用         280,470      217,831
8         預期信用減損(迴轉利益)損失        (32,244)          400
9                   利息費用         611,465      495,913
10                  利息收入       (770,955)    (794,577)
11                  股利收入       (226,618)    (181,318)
12                員工酬勞成本         381,133      838,895
13  採用權益法認列之關聯企業及合資利益之份額        (68,820)     (38,396)
14  處分及報廢不動產、廠房及設備損失(利益)          11,466     (36,519)
15             非金融資產減損損失         200,911            -
16                    其他         132,420      (4,746)
17              收益費損項目合計    

In [ ]:
table_dfs[0]

NameError: name 'table_dfs' is not defined

In [ ]:
file_path = '/content/drive/MyDrive/Colab Notebooks/2023表3.pdf'

table_list = camelot.read_pdf(file_path, pages="1", flavor="stream")
table_list[0]


<Table shape=(31, 14)>

In [ ]:
#aaron
# Get the table as a pandas DataFrame
table_df = table_list[0].df

# Print the DataFrame
print(table_df)

                       0  1           2          3           4            5   \
0                                   股　　本                                保留盈餘   
1                                                                              
2                                    普通股                    法定盈          特別盈   
3                                    股　本       資本公積         餘公積          餘公積   
4            民國一一一年一月一日餘額  $  18,197,753  1,726,136  14,842,179    2,682,297   
5                    本期淨利              -          -           -            -   
6                本期其他綜合損益              -          -           -            -   
7                本期綜合損益總額              -          -           -            -   
8                盈餘指撥及分配：                                                      
9                提列法定盈餘公積              -          -   2,050,808            -   
10                普通股現金股利              -          -           -            -   
11               特別盈餘公積迴轉              -

In [ ]:
file_path = '/content/drive/MyDrive/Colab Notebooks/2023表1.pdf'

table_list = camelot.read_pdf(file_path, pages="1", flavor="stream")
table_list[0]

<Table shape=(31, 14)>

In [ ]:
#aaron
# Get the table as a pandas DataFrame
table_df = table_list[0].df

# Print the DataFrame
print(table_df)

      0                                   1  2              3    4   \
0                                                    112.12.31        
1                                       資　　產              金　　額    %   
2                                      流動資產：                          
3   1100                   現金及約當現金(附註四及六(一))  $     17,656,357   14   
4   1110       透過損益按公允價值衡量之金融資產－流動(附註四及六(二))        14,446,307   11   
5   1170               應收票據及帳款淨額(附註四、六(三)及七)         9,731,631    8   
6   1200                   其他應收款(附註四、六(四)及七)           185,364    -   
7   130X                          存貨(附註六(五))        17,318,635   13   
8   1476                    其他金融資產－流動(附註四及八)            12,300    -   
9   1479                   其他流動資產－其他(附註六(十))         1,911,214    1   
10                                                  61,261,808   47   
11                                    非流動資產：                          
12  1517  透過其他綜合損益按公允價值衡量之金融資產－非流動(附註四及六(二))           811,937    1   
13  15

In [ ]:
file_path = '/content/drive/MyDrive/Colab Notebooks/2023表2.pdf'

table_list = camelot.read_pdf(file_path, pages="1", flavor="stream")
table_list[0]

<Table shape=(44, 7)>

In [ ]:
#aaron
# Get the table as a pandas DataFrame
table_df = table_list[0].df

# Print the DataFrame
print(table_df)

       0                             1  2            3    4            5    6
0                                                112年度             111年度     
1                                                  金　額    ％          金　額    ％
2   4000            營業收入(附註四、六(二十一)及七)  $  124,821,623  100  153,538,492  100
3   5000                  營業成本(附註六(五))     118,324,275   95  138,078,028   90
4                                 營業毛利       6,497,348    5   15,460,464   10
5                    營業費用：(附註六(二十二)及七)                                       
6   6100                          推銷費用       5,045,834    4    6,942,057    5
7   6200                          管理費用       2,145,332    2    2,231,793    1
8   6300                        研究發展費用       1,043,840    1    1,185,725    1
9   6450                  預期信用減損(利益)損失        (32,244)    -          400    -
10                                           8,202,762    7   10,359,975    7
11                             營業淨(損)利     (1,705,414)  (2)    5

In [ ]:
# shows list of top billionaires in 2023
table_dfs[0]

NameError: name 'table_dfs' is not defined

In [ ]:
# shows list of top billionaires
table_dfs[1]

,Year,Number of billionaires,Group's combined net worth
0,2023[2],"2,640",$12.2 trillion
1,2022[6],"2,668",$12.7 trillion
2,2021[11],"2,755",$13.1 trillion
3,2020,"2,095",$8.0 trillion
4,2019,"2,153",$8.7 trillion
5,2018,"2,208",$9.1 trillion
6,2017,"2,043",$7.7 trillion
7,2016,"1,810",$6.5 trillion
8,2015[18],"1,826",$7.1 trillion
9,2014[67],"1,645",$6.4 trillion


In [ ]:
# define query engines over these tables
df_query_engines = [PandasQueryEngine(table_df) for table_df in table_dfs]

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
response = df_query_engines[0].query(
    "What's the net worth of the second richest billionaire in 2023?"
)
print(str(response))

$180 billion


In [ ]:
response = df_query_engines[1].query("How many billionaires were there in 2009?")
print(str(response))

793


### Build Recursive Retriever

We define a top-level vector index that does top-k lookup over a set of Nodes. We define two special nodes (`IndexNode` objects) linking to each of these tables.

We define a `RecursiveRetriever` object to recursively retrieve/query nodes. We then put this in our `RetrieverQueryEngine` along with a `ResponseSynthesizer` to synthesize a response.

We pass in mappings from id to retriever and id to query engine. We then pass in a root id representing the retriever we query first.

In [ ]:
llm = OpenAI(temperature=0, model="gpt-4")

service_context = ServiceContext.from_defaults(
    llm=llm,
)

In [ ]:
doc_nodes = service_context.node_parser.get_nodes_from_documents(docs)

In [ ]:
# define index nodes
summaries = [
    "This node provides information about the world's richest billionaires in 2023",
    "This node provides information on the number of billionaires and their combined net worth from 2000 to 2023.",
]

df_nodes = [
    IndexNode(text=summary, index_id=f"pandas{idx}")
    for idx, summary in enumerate(summaries)
]

df_id_query_engine_mapping = {
    f"pandas{idx}": df_query_engine
    for idx, df_query_engine in enumerate(df_query_engines)
}

In [ ]:
# construct top-level vector index + query engine
vector_index = VectorStoreIndex(doc_nodes + df_nodes)
vector_retriever = vector_index.as_retriever(similarity_top_k=1)

In [ ]:
from llama_index.retrievers import RecursiveRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.response_synthesizers import get_response_synthesizer

recursive_retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever},
    query_engine_dict=df_id_query_engine_mapping,
    verbose=True,
)

response_synthesizer = get_response_synthesizer(
    # service_context=service_context,
    response_mode="compact"
)

query_engine = RetrieverQueryEngine.from_args(
    recursive_retriever, response_synthesizer=response_synthesizer
)

### Define Baseline Retriever

We also define a baseline retriever that does top-k lookup over the raw document.

In [ ]:
# baseline vector index (that doesn't include the extra df nodes).
# used to benchmark
vector_index0 = VectorStoreIndex(doc_nodes)
vector_query_engine0 = vector_index0.as_query_engine()

### Compare Results

We compare results between the recursive retriever vs. baseline retriever.

In [ ]:
response = query_engine.query(
    "How many billionaires were there in 2009?"
)
print(str(response))

Retrieving with query id None: How many billionaires were there in 2009?
Retrieved node with id, entering: pandas1
Retrieving with query id pandas1: How many billionaires were there in 2009?
Got response: 793
793


In [ ]:
response.source_nodes[0].node.get_content()

'Query: How many billionaires were there in 2009?\nResponse: 793'

In [ ]:
response = vector_query_engine0.query(
    "How many billionaires were there in 2009?"
)
print(str(response))

Based on the context information, it is not possible to determine the exact number of billionaires in 2009.


In [ ]:
print(response.source_nodes[1].node.get_content())

7/1/23, 11:31 PM
The World's Billionaires - Wikipedia
https://en.wikipedia.org/wiki/The_World%27s_Billionaires
6/33
In the 32nd annual Forbes list of the world's billionaires, the aggregate wealth of the top 20 richest
people on Earth amounted to about 13 percent of all billionaires' fortunes combined.[15] A record of
2,208 billionaires were in the ranking and the total wealth was $9.1 trillion, up 18% since 2017. For
the first time, Jeff Bezos was listed as the top billionaire due to Amazon's rising stock price that
resulted in one person's biggest one-year gain in wealth ($35 billion) since Forbes started tracking in
1987.[15] The U.S. had the most billionaires in the world, with 585, while China was catching up with
476 when including Hong Kong, Macau and Taiwan; it had 372 when excluding those three places.[15]
Forbes excluded Al-Walid bin Talal and all other Saudi billionaires due to the absence of accurate
wealth estimations as a result of the 2017–19 Saudi Arabian purge.[16]
No.

In [ ]:
response = query_engine.query(
    "What is the average age of top 5 billionaires in 2023? Make sure age is a float."
)
print(str(response))

Retrieving with query id None: What is the average age of top 5 billionaires in 2023? Make sure age is a float.
Retrieved node with id, entering: pandas0
Retrieving with query id pandas0: What is the average age of top 5 billionaires in 2023? Make sure age is a float.
Got response: 70.8
The average age of the top 5 billionaires in 2023 is 70.8.


In [ ]:
response = vector_query_engine0.query(
    "What is the average age of top 5 billionaires in 2023? Make sure age is a float."
)
print(str(response))

The average age of the top 5 billionaires in 2023 is 72.2.


Of course, just like the baseline vector query engine, the recursive retriever can answer semantic queries over the article as well.

In [ ]:
response = query_engine.query(
    "How is wealth accounted for in recipients if the billionaire is deceased?"
)
print(str(response))

Retrieving with query id None: How is wealth accounted for in recipients if the billionaire is deceased?
Retrieving text node: 7/1/23, 11:31 PM
The World's Billionaires - Wikipedia
https://en.wikipedia.org/wiki/The_World%27s_Billionaires
2/33
stock are priced to market on a date roughly a month before publication. Privately held companies are
priced by the prevailing price-to-sales or price-to-earnings ratios. Known debt is subtracted from
assets to get a final estimate of an individual's estimated worth in United States dollars. Since stock
prices fluctuate rapidly, an individual's true wealth and ranking at the time of publication may vary
from their situation when the list was compiled.[7]
When a living individual has dispersed his or her wealth to immediate family members it is included
under a single listing (as a single "family fortune") provided that individual (the grantor) is still living.
However, if a deceased billionaire's fortune has been dispersed, it will not appear as a

In [ ]:
response = vector_query_engine0.query(
    "How is wealth accounted for in recipients if the billionaire is deceased?"
)
print(str(response))

If a deceased billionaire's fortune has been dispersed, it will not appear as a single listing. Instead, each recipient will only appear if their own total net worth is over a billion dollars. In other words, the net worth of each individual recipient will not be combined with that of other family members.


## Extended Use Case: Parsing Tesla 10Qs

Here we deal with an even messier document format - parsing tables within a 10K.

We use the Unstructured library to help us extract tables.

The tables aren't perfectly formatted, but work well enough.

In [ ]:
!pip install unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.1/275.1 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 27.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 44.6 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993224 sha256=0bb73d4428e2d33042026c0aa9289324682e0cd7a4cf8d9edecc7f6a7bc1a81f
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


### Extract Elements

In [ ]:
from pydantic import BaseModel
from unstructured.partition.html import partition_html
import pandas as pd

In [ ]:
!wget "https://www.dropbox.com/scl/fi/mlaymdy1ni1ovyeykhhuk/tesla_2021_10k.htm?rlkey=qf9k4zn0ejrbm716j0gg7r802&dl=1" -O tesla_2021_10k.htm

--2023-11-08 15:51:33--  https://www.dropbox.com/scl/fi/mlaymdy1ni1ovyeykhhuk/tesla_2021_10k.htm?rlkey=qf9k4zn0ejrbm716j0gg7r802&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:601b:18::a27d:812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc8c2f9df597ecdae58e0a717b26.dl.dropboxusercontent.com/cd/0/inline/CHLjbU75m2bMyWOrogDeJxZxDTWRHX3aJdMvB30xWHi54AHyO71ak9byoVYTwIWB7wOons_Wnr_w5fqpNv-HwOW2IkWj5tg2_87-PbYhMwV2iF7xl7uD17ut9ulTnPH01ZY/file?dl=1# [following]
--2023-11-08 15:51:34--  https://uc8c2f9df597ecdae58e0a717b26.dl.dropboxusercontent.com/cd/0/inline/CHLjbU75m2bMyWOrogDeJxZxDTWRHX3aJdMvB30xWHi54AHyO71ak9byoVYTwIWB7wOons_Wnr_w5fqpNv-HwOW2IkWj5tg2_87-PbYhMwV2iF7xl7uD17ut9ulTnPH01ZY/file?dl=1
Resolving uc8c2f9df597ecdae58e0a717b26.dl.dropboxusercontent.com (uc8c2f9df597ecdae58e0a717b26.dl.dropboxusercontent.com)... 162.125.65.15, 2620:100:6022:15::a27d:4

In [ ]:
from llama_index.readers.file.flat_reader import FlatReader
from pathlib import Path

reader = FlatReader()
docs_2021 = reader.load_data(Path("tesla_2021_10k.htm"))

In [ ]:
from llama_index.node_parser import (
    UnstructuredElementNodeParser,
)

node_parser = UnstructuredElementNodeParser()

In [ ]:
import os
import pickle

if not os.path.exists("2021_nodes.pkl"):
    raw_nodes_2021 = node_parser.get_nodes_from_documents(docs_2021)
    pickle.dump(raw_nodes_2021, open("2021_nodes.pkl", "wb"))
else:
    raw_nodes_2021 = pickle.load(open("2021_nodes.pkl", "rb"))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
100%|██████████| 105/105 [21:36<00:00, 12.35s/it]


In [ ]:
base_nodes_2021, node_mappings_2021 = node_parser.get_base_nodes_and_mappings(
    raw_nodes_2021
)

In [ ]:
example_index_node = [b for b in base_nodes_2021 if isinstance(b, IndexNode)][
    20
]

# Index Node
print(
    f"\n--------\n{example_index_node.get_content(metadata_mode='all')}\n--------\n"
)
# Index Node ID
print(f"\n--------\nIndex ID: {example_index_node.index_id}\n--------\n")
# Referenceed Table
print(
    f"\n--------\n{node_mappings_2021[example_index_node.index_id].get_content()}\n--------\n"
)

### Build Recursive Retriever

In [ ]:
from llama_index.retrievers import RecursiveRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index import VectorStoreIndex, ServiceContext
from llama_index.llms import OpenAI

In [ ]:
llm = OpenAI(model="gpt-4-1106-preview")
service_context = ServiceContext.from_defaults(llm=llm)

# construct top-level vector index + query engine
vector_index = VectorStoreIndex(base_nodes_2021, service_context=service_context)
vector_retriever = vector_index.as_retriever(similarity_top_k=1)
vector_query_engine = vector_index.as_query_engine(similarity_top_k=1)

In [ ]:
from llama_index.retrievers import RecursiveRetriever

recursive_retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever},
    node_dict=node_mappings_2021,
    verbose=True,
)
query_engine = RetrieverQueryEngine.from_args(recursive_retriever)

### Run Some Queries

In [ ]:
response = query_engine.query("What was the revenue in 2020?")
print(str(response))

In [ ]:
# compare against the baseline retriever
response = vector_query_engine.query("What was the revenue in 2020?")
print(str(response))